Tool Binding : a process where u connect ur llm with a given tool


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
#Tool create
@tool #tool decorator
def multiply(a: int , b : int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a*b

In [4]:
print(multiply.invoke({"a":3,"b":5}))
print(multiply.name)
print(multiply.description)
print(multiply.args)

15
multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [8]:
#tool binding
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [9]:
llm_with_tools=llm.bind_tools([multiply]) # yeh meine tools ko bind krdiya agr ik se jyada ho toh , daalke hojega

TOOL CALLING 

In [55]:
llm_with_tools.invoke("Hi how are you")

AIMessage(content="I'm doing great! How can I help you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c6f71-5f30-79a1-ac2f-f301b14e166b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 58, 'output_tokens': 13, 'total_tokens': 71, 'input_token_details': {'cache_read': 0}})

In [56]:
ai_msg=llm_with_tools.invoke("Can you multiply 3 with 10")
result1=ai_msg.tool_calls
result2=llm_with_tools.invoke("Can you add 3 with 10").tool_calls


Tool Execution and
tool message


In [57]:
result1[0]['args']

{'b': 10, 'a': 3}

In [58]:
multiply.invoke(result1[0]['args'])
multiply.invoke(result1[0]) #this will give tool message

ToolMessage(content='30', name='multiply', tool_call_id='d73268fe-8aba-4675-87d6-552979c9105b')

In [59]:
multiply.invoke({'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'ae908a58-d80e-4b93-a78c-87e2b6d2ced7',
  'type': 'tool_call'})

ToolMessage(content='30', name='multiply', tool_call_id='ae908a58-d80e-4b93-a78c-87e2b6d2ced7')

In [79]:
query = HumanMessage(content="can you multiply 3 with 10")


In [80]:
messages = [query]

In [81]:
messages.append(ai_msg)

In [82]:
tool_msg =multiply.invoke(result1[0])

In [83]:
messages.append(tool_msg)

In [85]:
messages


[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 10, "a": 3}'}, '__gemini_function_call_thought_signatures__': {'d73268fe-8aba-4675-87d6-552979c9105b': 'CvcBAb4+9vvwS8DZ4pMkceD5XFhsGdBxTrhsukiNSQvZ7/CcDFzk37uWnZpvxOBsbaC5jOG9fzdtVmWlAcR58lrUBI6idg267ubi/pSNH5YGT1rKyuAnmFYdGFcQKfI+e/HQl6NzIRsAJZqMXHNKP+FAAvW/oOxaa3tProILCX/lmRet1nAIOPmcbdKJx/Al3nzJt7y81fuhsl8VuBrCOd7JL3NaOchWD0uJiD8utp2XIqyA63LLCdpHbQKGsN1/SnxaKjOAP4+eP2txFWvhI6lssq5JG2KFGq5srhxcTqEjZ/DYJQkxY8/OADBrqc3LmId0KgQ0I4n6VA=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c6f71-6846-73e3-8d64-a2e4e513d0df-0', tool_calls=[{'name': 'multiply', 'args': {'b': 10, 'a': 3}, 'id': 'd73268fe-8aba-4675-87d6-552979c9105b', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'i

In [86]:
final = llm_with_tools.invoke(messages)
print(final.content)

The product of 3 and 10 is 30.


The Entire FLow is User → LLM → Tool suggestion → Tool run → Result → LLM → Final answer
